<a href="https://colab.research.google.com/github/atick-faisal/TAVI/blob/main/src/inference/Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fix for GDrive

In [1]:
!pip install -U --no-cache-dir gdown --pre > /dev/null


# Mount GDrive

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# Download and Extract Dataset

In [3]:
!gdown "1Hkn-xh9gqjWOCkFA0OwzvWPDRsIPwdSv"
!unzip -o "AAA_DATASET_v7.zip" > /dev/null

Downloading...
From (uriginal): https://drive.google.com/uc?id=1Hkn-xh9gqjWOCkFA0OwzvWPDRsIPwdSv
From (redirected): https://drive.google.com/uc?id=1Hkn-xh9gqjWOCkFA0OwzvWPDRsIPwdSv&confirm=t&uuid=f3ad0633-aee7-4ac1-84da-c2385d2e9f65
To: /content/AAA_DATASET_v7.zip
100% 3.42G/3.42G [00:44<00:00, 76.3MB/s]


# Imports

In [40]:
import os
import datetime
import matplotlib
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm

# matplotlib.use('Agg')
plt.rcParams["font.size"] = 16

# Config

In [41]:
PROBLEM = "Curvature_2_ECAP"

MODEL_NAME = "MultiViewUNet_Curvature_2_TAWSS_2_ECAP"
DATASET_DIR = "/content/Images/"
TRAIN_DIR = "Train/"
TEST_DIR = "Test/"
INPUT_DIR = PROBLEM.split("_2_")[0]
TARGET_DIR = PROBLEM.split("_2_")[1]
MODEL_DIR = "/content/drive/MyDrive/Research/TAVI/Models/"
PRED_DIR = "/content/drive/MyDrive/Research/TAVI/Predictions/"
IMG_SIZE = 256
BATCH_SIZE = 16
BUFFER_SIZE = 1000
VAL_SPLIT = 0.2
LEARNING_RATE = 0.001
N_EPOCHS = 300
PATIENCE = 30

MODEL_PATH = f"{MODEL_DIR}/Curvature_2_TAWSS_MultiViewUNet_I256_B16_LR0.001/Apr-04-01:12AM"
SECOND_MODEL_PATH = f"{MODEL_DIR}/TAWSS_2_ECAP_MultiViewUNet_I256_B16_LR0.001/Apr-03-03:09PM"

EXP_NAME = f"{MODEL_NAME}"

# DataLoader

In [42]:
def load_data_from_dir(path: str) -> tf.data.Dataset:
    return tf.keras.utils.image_dataset_from_directory(
        directory=path,
        labels=None,
        color_mode='rgb',
        batch_size=BATCH_SIZE,
        image_size=(IMG_SIZE, IMG_SIZE),
        shuffle=False,
        seed=42,
        interpolation='bilinear',
        follow_links=False,
        crop_to_aspect_ratio=False
    )

# Load Dataset

In [43]:
trainX = load_data_from_dir(os.path.join(DATASET_DIR, TRAIN_DIR, INPUT_DIR))
trainY = load_data_from_dir(os.path.join(DATASET_DIR, TRAIN_DIR, TARGET_DIR))
testX = load_data_from_dir(os.path.join(DATASET_DIR, TEST_DIR, INPUT_DIR))
testY = load_data_from_dir(os.path.join(DATASET_DIR, TEST_DIR, TARGET_DIR))

train_ds = tf.data.Dataset.zip((trainX, trainY))
test_ds = tf.data.Dataset.zip((testX, testY))

print(train_ds.element_spec)
print(test_ds.element_spec)

Found 8208 files belonging to 1 classes.
Found 8208 files belonging to 1 classes.
Found 312 files belonging to 1 classes.
Found 312 files belonging to 1 classes.
(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None))
(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None))


# Normalization

In [44]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (
    normalization_layer(x), normalization_layer(y)))
test_ds = test_ds.map(lambda x, y: (
    normalization_layer(x), normalization_layer(y)))

# Optimization

In [45]:
AUTOTUNE = tf.data.AUTOTUNE

train_batches = (
    train_ds
    .cache()
    .shuffle(BUFFER_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

test_batches = (
    test_ds
    .cache()
    .prefetch(buffer_size=AUTOTUNE)
)

# Loss Functions / Metrics

In [46]:
def attention_mse(y_true, y_pred):
    _y_true = y_true[y_true != 1.0]
    _y_pred = y_pred[y_true != 1.0]
    squared_difference = tf.square(_y_true - _y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)


def attention_mae(y_true, y_pred):
    _y_true = y_true[y_true != 1.0]
    _y_pred = y_pred[y_true != 1.0]
    squared_difference = tf.abs(_y_true - _y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)

# Load Models

In [47]:
model = tf.keras.models.load_model(
    MODEL_PATH,
    custom_objects={
        "attention_mse": attention_mse,
        "attention_mae": attention_mae
    }
)
second_model = tf.keras.models.load_model(
    SECOND_MODEL_PATH,
    custom_objects={
        "attention_mse": attention_mse,
        "attention_mae": attention_mae
    }
)


# Prediction

In [48]:
timestamp = datetime.datetime.now().strftime('%b-%d-%I:%M%p')


In [49]:
pred_path = os.path.join(PRED_DIR, EXP_NAME, timestamp)
if not os.path.exists(pred_path):
    os.makedirs(pred_path)

emae, emse = 0.0, 0.0

for idx, (input, target) in enumerate(test_batches.unbatch()):
    prediction = tf.squeeze(
        model.predict(
                tf.expand_dims(input, axis=0),
                verbose=0
            )
        ).numpy()
    
    prediction[
        tf.math.reduce_all((target == [1.0, 1.0, 1.0]), axis=-1)
    ] = [1.0, 1.0, 1.0]

    second_prediction = tf.squeeze(
        second_model.predict(
                tf.expand_dims(prediction, axis=0),
                verbose=0
            )
        ).numpy()

    second_prediction[
        tf.math.reduce_all((target == [1.0, 1.0, 1.0]), axis=-1)
    ] = [1.0, 1.0, 1.0]

    loss, acc = second_model.evaluate(
        tf.expand_dims(prediction, axis=0),
        tf.expand_dims(target, axis=0)
    )

    emae = emae + acc
    emse = emse + loss

    plt.figure(figsize=(7, 7))
    plt.imshow(target)
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(os.path.join(pred_path, f"{idx}_T.png"))
    plt.close()

    plt.figure(figsize=(7, 7))
    plt.imshow(second_prediction)
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(os.path.join(pred_path, f"{idx}_P.png"))
    plt.close()

print(f"EMAE: {emae / (idx + 1)}")
print(f"EMSE: {emse / (idx + 1)}")

1/1 [==============================] - 0s 47ms/step - loss: 0.0391 - attention_mae: 0.1152
EMAE: 0.13379853134019634
EMSE: 0.05056985643787835
